In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
!pip install rouge
from rouge import Rouge
ROUGE = Rouge()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stopWords = set(stopwords.words("english"))
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#lemma
def lemma(text:str) -> str:
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens] 
    lemma_word = []
    wordnet_lemmatizer = WordNetLemmatizer()
    for w in tokens:
        word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
        word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
        word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
        lemma_word.append(word3)
    preprocessed_text = ' '.join(lemma_word)

    return preprocessed_text

# Load Dataset

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 10.6 MB/s 
     |████████████████████████████████| 140 kB 58.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.5 MB/s 
     |████████████████████████████████| 212 kB 69.2 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 271 kB 51.5 MB/s 
     |████████████████████████████████| 144 kB 26.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 112 kB 11.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing in

In [ ]:
import os
import datasets
import pandas as pd

dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')

dataset_path = os.path.join(".", "cnn_daily_ds")
if os.path.exists(dataset_path):
  print("exist path")
  #raise Exception(f"{dataset_path} exists")
else:
  os.mkdir(dataset_path)

print("Convert test dataset to csv")
test_data = dataset['test']
test_data.to_csv(os.path.join(dataset_path, "test.csv"))
test_data = pd.read_csv(os.path.join(dataset_path, "test.csv"))
test_data = test_data.iloc[:5000]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Convert test dataset to csv


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

# Method 1: Easy WordCount 

In [ ]:
Rouge_TotalScore_1gram = 0
Rouge_TotalScore_2gram = 0
Rouge_TotalScore_Lgram = 0
Bleu_TotalScore = 0
epoch = 1
print("100 pieces of data per epoch ")
for i in range(test_data.shape[0]):
  if(i >= (epoch*100)):
    print(f"epoch {epoch} complete")
    epoch+=1

  text = test_data['article'].iloc[i]
  reference = test_data['highlights'].iloc[i]
  #text_ps = ps.stem(text)
  #text_lemma = lemma(text)

  words = word_tokenize(text)
  sentences = sent_tokenize(text)

  freqTable = dict()
  for word in words:
    word = word.lower()
    word = lemma(word)
    if word in stopWords:
      continue
    if word in freqTable:
      freqTable[word] += 1
    else:
      freqTable[word] = 1

  sentenceValue = dict()
  for sentence in sentences:
    for word, freq in freqTable.items():
      if word in sentence.lower():
        if sentence in sentenceValue:
          sentenceValue[sentence] += freq
        else:
          sentenceValue[sentence] = freq

  sumValues = 0
  for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

  average = int(sumValues / len(sentenceValue))

  summary_EasyWordCount = ''
  for sentence in sentences:
    if(sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
      summary_EasyWordCount += " " + sentence

  if(summary_EasyWordCount == ''):
    Rouge_ScoreValue = 0
    Bleu_ScoreValue = 0
  else:
    Rouge_Score = ROUGE.get_scores(summary_EasyWordCount, reference)  
    Rouge_ScoreValue_1gram = Rouge_Score[0]['rouge-1']['r']
    Rouge_ScoreValue_2gram = Rouge_Score[0]['rouge-2']['r']
    Rouge_ScoreValue_Lgram = Rouge_Score[0]['rouge-l']['r']
    Bleu_ScoreValue = sentence_bleu(reference, summary_EasyWordCount)

  Rouge_TotalScore_1gram += Rouge_ScoreValue_1gram
  Rouge_TotalScore_2gram += Rouge_ScoreValue_2gram
  Rouge_TotalScore_Lgram += Rouge_ScoreValue_Lgram
  Bleu_TotalScore += Bleu_ScoreValue


Rouge_Score_1gram_EasyWordcount = Rouge_TotalScore_1gram / test_data.shape[0]
Rouge_Score_2gram_EasyWordcount = Rouge_TotalScore_2gram / test_data.shape[0]
Rouge_Score_Lgram_EasyWordcount = Rouge_TotalScore_Lgram / test_data.shape[0]
Bleu_Score_EasyWordcount = Bleu_TotalScore / test_data.shape[0]
print("Rouge 1-gram: ", Rouge_Score_1gram_EasyWordcount)
print("Rouge 2-gram: ", Rouge_Score_2gram_EasyWordcount)
print("Rouge L-gram: ", Rouge_Score_Lgram_EasyWordcount)
print("Bleu: ", Bleu_Score_EasyWordcount)

100 pieces of data per epoch 
epoch 1 complete
epoch 2 complete
epoch 3 complete
epoch 4 complete
epoch 5 complete
epoch 6 complete
epoch 7 complete
epoch 8 complete
epoch 9 complete
epoch 10 complete
epoch 11 complete
epoch 12 complete
epoch 13 complete
epoch 14 complete
epoch 15 complete
epoch 16 complete
epoch 17 complete
epoch 18 complete
epoch 19 complete
epoch 20 complete
epoch 21 complete
epoch 22 complete
epoch 23 complete
epoch 24 complete
epoch 25 complete
epoch 26 complete
epoch 27 complete
epoch 28 complete
epoch 29 complete
epoch 30 complete
epoch 31 complete
epoch 32 complete
epoch 33 complete
epoch 34 complete
epoch 35 complete
epoch 36 complete
epoch 37 complete
epoch 38 complete
epoch 39 complete
epoch 40 complete
epoch 41 complete
epoch 42 complete
epoch 43 complete
epoch 44 complete
epoch 45 complete
epoch 46 complete
epoch 47 complete
epoch 48 complete
epoch 49 complete
Rouge 1-gram:  0.5406401393155483
Rouge 2-gram:  0.21622087607096596
Rouge L-gram:  0.50248905386

# Method 2: Bert Extractive

In [ ]:
!pip install bert-extractive-summarizer
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 36.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 29.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9d2266a4630c00fc1aaba681d4469bc03e10530805b25c57058f342246bb9961
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
from summarizer import Summarizer
model = Summarizer()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
Rouge_TotalScore_1gram = 0
Rouge_TotalScore_2gram = 0
Rouge_TotalScore_Lgram = 0
Bleu_TotalScore = 0
epoch = 1
print("100 pieces of data per epoch ")
for i in range(test_data.shape[0]):
  if(i >= (epoch*100)):
    print(f"epoch {epoch} complete")
    epoch+=1

  text = test_data['article'].iloc[i]
  reference = test_data['highlights'].iloc[i]

  result = model(text)
  summary_Bert = "".join(result)

  #caculate average count
  if(summary_EasyWordCount == ''):
    Rouge_ScoreValue = 0
    Bleu_ScoreValue = 0
  else:
    Rouge_Score = ROUGE.get_scores(summary_Bert, reference)  
    Rouge_ScoreValue_1gram = Rouge_Score[0]['rouge-1']['r']
    Rouge_ScoreValue_2gram = Rouge_Score[0]['rouge-2']['r']
    Rouge_ScoreValue_Lgram = Rouge_Score[0]['rouge-l']['r']
    Bleu_ScoreValue = sentence_bleu(reference, summary_Bert)

  Rouge_TotalScore_1gram += Rouge_ScoreValue_1gram
  Rouge_TotalScore_2gram += Rouge_ScoreValue_2gram
  Rouge_TotalScore_Lgram += Rouge_ScoreValue_Lgram
  Bleu_TotalScore += Bleu_ScoreValue


Rouge_Score_1gram_Bert = Rouge_TotalScore_1gram / test_data.shape[0]
Rouge_Score_2gram_Bert = Rouge_TotalScore_2gram / test_data.shape[0]
Rouge_Score_Lgram_Bert = Rouge_TotalScore_Lgram / test_data.shape[0]
Bleu_Score_Bert = Bleu_TotalScore / test_data.shape[0]
print("Rouge 1-gram: ", Rouge_Score_1gram_Bert)
print("Rouge 2-gram: ", Rouge_Score_2gram_Bert)
print("Rouge L-gram: ", Rouge_Score_Lgram_Bert)
print("Bleu: ", Bleu_Score_Bert)

100 pieces of data per epoch 
epoch 1 complete
epoch 2 complete
epoch 3 complete
epoch 4 complete
epoch 5 complete
epoch 6 complete
epoch 7 complete
epoch 8 complete
epoch 9 complete
epoch 10 complete
epoch 11 complete
epoch 12 complete
epoch 13 complete
epoch 14 complete
epoch 15 complete
epoch 16 complete
epoch 17 complete
epoch 18 complete
epoch 19 complete
epoch 20 complete
epoch 21 complete
epoch 22 complete
epoch 23 complete
epoch 24 complete
epoch 25 complete
epoch 26 complete
epoch 27 complete
epoch 28 complete
epoch 29 complete
epoch 30 complete
epoch 31 complete
epoch 32 complete
epoch 33 complete
epoch 34 complete
epoch 35 complete
epoch 36 complete
epoch 37 complete
epoch 38 complete
epoch 39 complete
epoch 40 complete
epoch 41 complete
epoch 42 complete
epoch 43 complete
epoch 44 complete
epoch 45 complete
epoch 46 complete
epoch 47 complete
epoch 48 complete
epoch 49 complete
Rouge 1-gram:  0.45651426792329247
Rouge 2-gram:  0.16594448926323513
Rouge L-gram:  0.4217944279